# Monetary Transmission Mechanism

## Introduction
https://medium.com/@yuhui_w/monetary-transmission-mechanism-vector-autoregression-var-approach-d4dd87cfe435  
   

[FRED-MD data](https://research.stlouisfed.org/econ/mccracken/fred-databases/)  
[FRED-MD Paper & Columns](https://s3.amazonaws.com/real.stlouisfed.org/wp/2015/2015-012.pdf)

In [1]:
# data
import pandas as pd
import pandas_profiling
import numpy as np
import datetime as dt

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
from statsmodels.tsa.api import VAR, VARMAX

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pandas_datareader.data as web
import yfinance as yf

In [2]:
# download CSI300 data from yahoo finance
ticker = '3188.hk'
start = str(dt.datetime(2010, 1, 1))
end = str(dt.datetime.now())
ticker = web.DataReader(ticker, 'yahoo', start, end).Close

# resample to monthly
ticker = ticker.resample('MS').first()
ticker.head()


Date
2012-07-01    24.250000
2012-08-01    23.389999
2012-09-01    22.360001
2012-10-01    22.750000
2012-11-01    28.850000
Freq: MS, Name: Close, dtype: float64

In [5]:
# download currency data hkdusd
fx = 'HKD=X'
fx = web.DataReader(fx, 'yahoo', start, end).Close
# resample to monthly
fx = fx.resample('MS').first()
fx.head()

Date
2009-12-01    7.7537
2010-01-01    7.7529
2010-02-01    7.7676
2010-03-01    7.7638
2010-04-01    7.7677
Freq: MS, Name: Close, dtype: float64

In [27]:
trade = pd.concat([ticker, fx], axis=1)
trade = trade.dropna()
trade.head()

,Close,Close
Date,,
2012-07-01,24.250000,7.7572
2012-08-01,23.389999,7.7550
2012-09-01,22.360001,7.7559
2012-10-01,22.750000,7.7542
2012-11-01,28.850000,7.7499


In [28]:
data = pd.read_csv('current.csv')


# format date to year-month-day
data['sasdate'] = pd.to_datetime(data['sasdate'],errors ='coerce')
data['Date'] = data['sasdate'].dt.strftime('%Y-%m-%d')
data.set_index('Date', inplace = True)

data = data.iloc[1:,1:]
data.head()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
Date,,,,,,,,,,,,,,,,,,,,,
1959-01-01,2442.158,2293.2,17.272,292266.4261,18235.77392,22.0733,23.4027,22.2858,31.5688,18.6601,...,17.791,11.326,2.13,2.45,2.04,NaN,6476.0,12298.0,84.2043,NaN
1959-02-01,2451.778,2301.5,17.452,294424.7425,18369.56308,22.5056,23.7185,22.4788,31.8004,18.7580,...,17.798,11.343,2.14,2.46,2.05,NaN,6476.0,12298.0,83.5280,NaN
1959-03-01,2467.594,2318.5,17.617,293418.6704,18523.05762,22.8298,23.8621,22.5892,31.8004,19.1254,...,17.785,11.363,2.15,2.45,2.07,NaN,6508.0,12349.0,81.6405,NaN
1959-04-01,2483.671,2334.9,17.553,299322.8039,18534.46600,23.3161,24.2067,22.9201,32.3021,19.2478,...,17.796,11.403,2.16,2.47,2.08,NaN,6620.0,12484.0,81.8099,NaN
1959-05-01,2498.026,2350.4,17.765,301364.3249,18679.66354,23.6673,24.4077,23.1408,32.4565,19.6396,...,17.777,11.421,2.17,2.48,2.08,95.3,6753.0,12646.0,80.7315,NaN


In [31]:
# merge ticker n fx to data 
data = pd.concat([data, trade], axis=1)
data = data.dropna()
data.head()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:2618: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'float', sort order is undefined for incomparable objects
  return Index.union(this, other, sort=sort).astype(object, copy=False)


,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx,Close,Close,Close,Close
Date,,,,,,,,,,,,,,,,,,,,,


In [ ]:
# check for missing values
print(data.isnull().sum().sort_values(ascending=False))
data = data.iloc[:-1,:]

In [ ]:
# ffill null values and drop column with the most null values
data = data.fillna(method='ffill')
to_drop = data.columns[data.isnull().sum() != 0].sort_values(ascending=False)
print(to_drop)

In [ ]:
# drop columns in to_drop
# data = data.drop(to_drop, axis=1)
# data.head()
data = data.dropna(axis=0)

In [ ]:
data['fx']

## Feature Selection 


In [ ]:
# target variable
target_var = 'FEDFUNDS'

from sklearn.feature_selection import SelectKBest, f_regression
X_train, y_train = data.loc[:, data.columns != target_var], data[target_var]

selector = SelectKBest(f_regression, k=15)
selector.fit(X_train, y_train)
scores = -np.log(selector.pvalues_)


X_indices = np.arange(X_train.shape[-1])
plt.figure(1)
plt.clf()
plt.bar(x = X_indices, height = scores, width=0.2)
plt.title("Feature univariate score")
plt.xlabel("Feature number")
plt.ylabel(r"Univariate score ($-Log(p_{value})$)")
plt.show()

In [ ]:
score_df = pd.DataFrame([data.columns, scores]).T
score_df.columns = ['Feature', 'Score']
score_df = score_df.sort_values(by='Score', ascending=False)
score_df.head(20)

## Working only with the varibales of interest

In [ ]:
# get the relevant columns
endo = ['FEDFUNDS', 'INDPRO', 'TB3MS', 'PAYEMS', 'HOUST', 'DPCERA3M086SBEA', 'CPIAUCSL', 'M2REAL', 'BUSLOANS', 'S&P 500', 'S&P PE ratio', 'fx', 'asset']

data = data[endo]

### Cointegration


In [ ]:
# function to calculate the cointegration matrix
# organised pvalues into matrix
def coint_matrix(data):
    coint_matrix = pd.DataFrame(index=data.columns, columns=data.columns)
    for i in data.columns:
        for j in data.columns:
            if i != j:
                x = data[i]
                y = data[j]
                result = sm.tsa.stattools.coint(x, y)
                coint_matrix.loc[i,j] = result[1]
    # fill nan will 0
    coint_matrix = coint_matrix.fillna(0)
    return coint_matrix

In [ ]:
coint = coint_matrix(data)

# plot a heatmap of the cointegration matrix
plt.subplots(figsize=(15,15))
sns.heatmap(coint, annot=True)


Statsmdoels uses augmented Engle-Granger test for cointegration.    
   
The Null hypothesis is that there is no cointegration,   
the alternative hypothesis is that there is cointegrating relationship.   
If the pvalue is small, below a critical size, then we can reject the hypothesis that there is no cointegrating relationship.    
    
System-wide cointegration does not imply pairwise cointegration, although pairwise cointegration does imply system-wide cointegration https://stats.stackexchange.com/questions/171066/cointegration-between-more-than-two-variables    
   
From the above cointegration p-value heatmap, we see that new private housing and PE ratio are not cointegrated with the other series. As a result we are dropping these two series from our model.    
We are retaining Fed fund rate as it is the key pathway to manage monetary policy.

In [ ]:
# drop the columns with pvalue > 0.05
data = data.drop(['HOUST', 'S&P PE ratio'], axis=1)

### Transformation & Stationarity

In [ ]:
# Transformation required
T2 = ['FEDFUNDS', 'TB3MS']
T5 = ['INDPRO', 'PAYEMS', 'DPCERA3M086SBEA', 'M2REAL', 'S&P 500']
T6 = ['CPIAUCSL', 'BUSLOANS']

data[T2] = data[T2].diff()
data[T5] = np.log(data[T5]).diff()
data[T6] = np.log(data[T6]).diff()**2

data.dropna(inplace=True)
data.head()

In [ ]:
# check for stationarity
data.plot(figsize=(20,10))

In [ ]:
# acf & pacf
for column in data.columns:
    print(column)
    fig, ax = plt.subplots(1,2,figsize=(10,5))
    sm.graphics.tsa.plot_acf(data[column], lags=20, ax=ax[0])
    sm.graphics.tsa.plot_pacf(data[column], lags=12, alpha=0.05, ax=ax[1])
    plt.show()


In [ ]:
# stationarity test
output = []
for column in data.columns:
    pval = tsa.adfuller(data[column], regression='c', autolag='AIC')[1]
    output.append(pval)
output = pd.DataFrame(output).T
output.columns = data.columns
odd = output.columns[output.max() >= 0.05]
output[odd]

In [ ]:
data.plot()

# VAR model 


In [ ]:
maxlag = 12
VAR_model = VAR(data) 
VAR_model.select_order(maxlag)
results = VAR_model.fit(maxlags=maxlag, ic='aic')
results.summary()

In [ ]:
lag_order = results.k_ar
lag_order

In [ ]:
irf = results.irf(maxlag)
irf.plot(orth=False, figsize=(50, 50))

In [ ]:
irf.plot_cum_effects(impulse = 'FEDFUNDS', orth=False, figsize=(7, 30))

In [ ]:
irf.plot_cum_effects(response = 'CPIAUCSL', orth=False, figsize=(7, 30))

In [ ]:
# forecast error varince decomposition
fevd = results.fevd(lag_order)

fevd.summary()